# not realized


In [0]:
def fix_strange_letters(big_letters, image, chars):
  for (x,y,w,h) in big_letters:
    big_letter = image[y:y+h, x:x+w]
    ret, temp = cv2.threshold(big_letter,120,255,cv2.THRESH_BINARY)
    plot_image(temp, 'gray')
    for (char, char_img)  in chars.items():
      h1, w1 = char_img.shape
      temp = deepcopy(big_letter)
      temp = cv2.resize(temp, (int(w/1.35), int(h/1.35)), interpolation = cv2.INTER_AREA)
      
      feature_map = cv2.matchTemplate(temp, char_img, cv2.TM_CCOEFF_NORMED)
      min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(feature_map)
      top_left = max_loc
      bottom_right = (top_left[0] + h1, top_left[1] + w1)
      
      cv2.rectangle(temp, top_left, bottom_right, 0, 1)
      plot_image(temp, 'gray')
      print('key: {}, maxval: {}'.format(char, max_val))
    assert 0

    
coordinates = sorted(coordinates, key=lambda ctr: ctr[0] + ctr[1]//25 * gray_image.shape[1])


def concat_bboxes(coordinates):
  new_coordinates = []
  tmp_coords = coordinates[0]
  i = 1
  while i < len(coordinates):
    x,y,w,h = coordinates[i]
    space_between = abs(tmp_coords[0]+tmp_coords[2] - x)
    if space_between < 15:
      x1 = tmp_coords[0] # left x
      y1 = min(y, tmp_coords[1]) # for max height
      w1 = w + tmp_coords[2] + space_between #  prew_w + next_w + between_w 
      h1 = max(h, tmp_coords[3]) # max h
      tmp_coords = (x1,y1,w1,h1)
      i+=1
    else:
      new_coordinates.append(tmp_coords)
      tmp_coords = (x,y,w,h)
      i+=1
  new_coordinates.append(tmp_coords) # last
  return new_coordinates


  
# batch_size - розмір батчу
# num_samples - кількість слів (число кратне batch_size)
# alphabet_str[num_classes] - алфавіт (lower+upper+digits+.) 
#
# all_pred_words [num_samples, ?] - сампли з передбаченими словами
# all_true_words [num_samples, word_lens] - сампли з правдивими словами
def get_all_predictions(batch_size, num_samples, alphabet_str):
  print('start')
  all_pred_words = []
  all_true_words = []
  for i, data in enumerate(data_gen(train=0)): # should use batch_size = 1 
    inputs, outputs = data
    batch_x, batch_y, word_lens = inputs['the_input'], inputs['the_labels'], inputs['label_length']
    y_pred = model.predict_on_batch(batch_x)
    
    pred_words, true_words = get_words_from_y(y_pred, batch_y, word_lens, alphabet_str)
    all_pred_words += pred_words
    all_true_words += true_words
    if i >= int(num_samples/batch_size):
      break
    return all_pred_words, all_true_words

  
  
# генерує дані для входу і виходу молелі
def data_gen(train=1):
  if not getattr(data_gen, 'attrs', 0):
      data_gen.attrs = True
      data_gen.font_names = [i for i in font_paths.keys() if i not in late_paths]
      data_gen.batch_size = 64
      data_gen.max_word_len = 16
      data_gen.font_p = None
      data_gen.train_dir_path = r'/content/drive/My Drive/Colab Notebooks/course_project/data_crnn/train/x'
      data_gen.test_dir_path = r'/content/drive/My Drive/Colab Notebooks/course_project/data_crnn/test/x'
  while True:
    if train:
      data_dir_path = data_gen.train_dir_path
    else:
      data_dir_path = data_gen.test_dir_path
    x_batch, y_batch, input_lens, word_lens = get_batch(data_dir_path, data_gen.font_names, data_gen.batch_size, data_gen.max_word_len, data_gen.font_p)
    inputs = {
        'the_input': x_batch, 
        'the_labels': y_batch,
        'input_length': input_lens,
        'label_length': word_lens
    }
    outputs = {'ctc': np.zeros([data_gen.batch_size])} # не юзається бо лосс функція не юзає output, а юзає лейбли з інпута
    yield (inputs, outputs)